# pygohome

## Python, Let's Go Home. Quickly.

*pygohome* is a 100% personal route optimizer in a known environment based on experience.

*You* walk/ride/drive frequently between known locations (home, work, school, shops, family, friends, …) using different routes, but would like to know the optimal route, that should take you the less time possible? *pygohome* uses your recorded GPX tracks to build a route network of *your* world with estimation on how long *you* need to get from A to B using the mean of transport of your choice.

## How it works

1. **Choose your mean of transport.** Bicycle works great, walking should too. Motorized vehicles may be served better by real-time online services.

2. **Track all your trips** using that mean of transport. Start the tracking before you leave, stop the tracking after you arrive. Ride/walk/drive normally, stop at lights, don't speed. OsmAnd works great (list of other apps needed). 1 or 2 seconds tracking interval is ok.

3. **Add POIs (and intersections) as waypoints.** Transfer all GPX files to your computer. Load all of them into some GPX editor (JOSM works great).

4. **Create a new GPX file with waypoints** and add all points of interest (home, work, or any place where you started, deliberately paused or ended a trip) with an attribute `name=…`. Then add unnamed waypoints on each intersection where your tracks cross, split, or join. This is the biggest future plan in *pygohome* to offer a good editor with checks for consistency. Now you have to do it manually.

5. **Run the cell below** and an empty map will appear. All the action happens here from now on.

6. **Load your GPX files** with the tracks AND the file with waypoints. The current limit is 10MB for one upload, so if you have more data, do it in more batches.

7. **Find the fastest route** where `src` and `dst` are the names of the POIs and `Quantile` is the a float number between `0.0` (take into account your best time on each road segment) and `1.0` (worst time), while `0.8` is considered a safe value.

In [ ]:
from pygohome.jupygohome import gohome
gohome()